# 이미지 대 이미지

이미지 대 이미지는 [텍스트 대 이미지](https://huggingface.co/docs/diffusers/main/en/using-diffusers/conditional_image_generation)와 유사하지만 프롬프트 외에도 확산 프로세스의 시작점으로 초기 이미지를 전달할 수도 있습니다. 초기 이미지는 잠재 공간으로 인코딩되고 노이즈가 추가됩니다. 그런 다음 잠재 확산 모델은 프롬프트와 노이즈가 있는 잠재 이미지를 가져와 추가된 노이즈를 예측하고 초기 잠재 이미지에서 예측된 노이즈를 제거하여 새 잠재 이미지를 얻습니다. 마지막으로 디코더는 새 잠재 이미지를 다시 이미지로 디코딩합니다.

🤗 Diffusers를 사용하면 다음과 같이 1-2-3단계로 쉽게 수행할 수 있습니다.

1. 체크포인트를 [AutoPipelineForImage2Image](https://huggingface.co/docs/diffusers/main/en/api/pipelines/auto_pipeline#diffusers.AutoPipelineForImage2Image) 클래스로 로드합니다. 이 파이프라인은 체크포인트를 기반으로 올바른 파이프라인 클래스 로드를 자동으로 처리합니다.

In [ ]:
import torch
from diffusers import AutoPipelineForImage2Image
from diffusers.utils import load_image, make_image_grid

pipeline = AutoPipelineForImage2Image.from_pretrained(
    "kandinsky-community/kandinsky-2-2-decoder", torch_dtype=torch.float16, use_safetensors=True
)
pipeline.enable_model_cpu_offload()
# xFormers가 설치되어 있지 않거나 PyTorch 2.0 이상이 설치되어 있는 경우 다음 줄을 제거합니다.
pipeline.enable_xformers_memory_efficient_attention()

<Tip>

가이드 전체에서 메모리를 절약하고 추론 속도를 높이기 위해 [enable_model_cpu_offload()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline.enable_model_cpu_offload) 및 [enable_xformers_memory_efficient_attention()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline.enable_xformers_memory_efficient_attention)을 사용하는 것을 알 수 있습니다. PyTorch 2.0을 사용하는 경우 파이프라인에서 [enable_xformers_memory_efficient_attention()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline.enable_xformers_memory_efficient_attention)을 호출할 필요가 없습니다. 이미 PyTorch 2.0의 기본 [스케일드 닷 프로덕트 어텐션](https://huggingface.co/docs/diffusers/main/en/using-diffusers/../optimization/fp16#scaled-dot-product-attention)을 사용하기 때문입니다.

</Tip>

2. 파이프라인에 전달할 이미지를 로드합니다.

In [ ]:
init_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/cat.png")

3. 파이프라인에 프롬프트와 이미지를 전달하여 이미지를 생성합니다.

In [ ]:
prompt = "cat wizard, gandalf, lord of the rings, detailed, fantasy, cute, adorable, Pixar, Disney, 8k"
image = pipeline(prompt, image=init_image).images[0]
make_image_grid([init_image, image], rows=1, cols=2)

<div class="flex gap-4">
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/cat.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">초기 이미지</figcaption>
  </div>
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">생성된 이미지</figcaption>
  </div>
</div>

## 인기 모델

가장 인기 있는 이미지-이미지 모델은 [Stable Diffusion v1.5](https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-v1-5), [Stable Diffusion XL (SDXL)](https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0) 및 [Kandinsky 2.2](https://huggingface.co/kandinsky-community/kandinsky-2-2-decoder)입니다. Stable Diffusion 및 Kandinsky 모델의 결과는 아키텍처 차이와 학습 프로세스로 인해 다릅니다. 일반적으로 SDXL은 Stable Diffusion v1.5보다 고품질 이미지를 생성할 것으로 예상할 수 있습니다. 각 모델을 사용하는 방법을 간략히 살펴보고 결과를 비교해 보겠습니다.

### 스테이블 디퓨전 v1.5

Stable Diffusion v1.5는 이전 체크포인트에서 초기화된 잠재 확산 모델이며, 512x512 이미지에서 595K 단계 동안 추가로 미세 조정되었습니다. 이 파이프라인을 이미지-이미지 변환에 사용하려면 파이프라인에 전달할 초기 이미지를 준비해야 합니다. 그런 다음 프롬프트와 이미지를 파이프라인에 전달하여 새 이미지를 생성할 수 있습니다.

In [ ]:
import torch
from diffusers import AutoPipelineForImage2Image
from diffusers.utils import make_image_grid, load_image

pipeline = AutoPipelineForImage2Image.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
pipeline.enable_model_cpu_offload()
# xFormers가 설치되어 있지 않거나 PyTorch 2.0 이상이 설치되어 있는 경우 다음 줄을 제거합니다.
pipeline.enable_xformers_memory_efficient_attention()

# 이미지 준비
url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img-init.png"
init_image = load_image(url)

prompt = "Astronaut in a jungle, cold color palette, muted colors, detailed, 8k"

# 파이프라인에 프롬프트와 이미지 전달
image = pipeline(prompt, image=init_image).images[0]
make_image_grid([init_image, image], rows=1, cols=2)

<div class="flex gap-4">
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img-init.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">초기 이미지</figcaption>
  </div>
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img-sdv1.5.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">생성된 이미지</figcaption>
  </div>
</div>

### 스테이블 디퓨전 XL (SDXL)

SDXL은 Stable Diffusion 모델의 더 강력한 버전입니다. 더 큰 기본 모델과 기본 모델의 출력을 더욱 향상시키는 추가 리파이너 모델을 사용합니다. 이 모델을 사용하는 방법과 고품질 이미지를 생성하는 데 사용하는 다른 기술에 대한 자세한 내용은 [SDXL](https://huggingface.co/docs/diffusers/main/en/using-diffusers/sdxl) 가이드를 참조하십시오.

In [ ]:
import torch
from diffusers import AutoPipelineForImage2Image
from diffusers.utils import make_image_grid, load_image

pipeline = AutoPipelineForImage2Image.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
pipeline.enable_model_cpu_offload()
# xFormers가 설치되어 있지 않거나 PyTorch 2.0 이상이 설치되어 있는 경우 다음 줄을 제거합니다.
pipeline.enable_xformers_memory_efficient_attention()

# 이미지 준비
url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img-sdxl-init.png"
init_image = load_image(url)

prompt = "Astronaut in a jungle, cold color palette, muted colors, detailed, 8k"

# 파이프라인에 프롬프트와 이미지 전달
image = pipeline(prompt, image=init_image, strength=0.5).images[0]
make_image_grid([init_image, image], rows=1, cols=2)

<div class="flex gap-4">
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img-sdxl-init.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">초기 이미지</figcaption>
  </div>
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img-sdxl.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">생성된 이미지</figcaption>
  </div>
</div>

### 칸딘스키 2.2

Kandinsky 모델은 Stable Diffusion 모델과 다릅니다. 이미지 사전 모델을 사용하여 이미지 임베딩을 생성하기 때문입니다. 임베딩은 텍스트와 이미지 간의 더 나은 정렬을 만드는 데 도움이 되며, 이를 통해 잠재 확산 모델이 더 나은 이미지를 생성할 수 있습니다.

Kandinsky 2.2를 사용하는 가장 쉬운 방법은 다음과 같습니다.

In [ ]:
import torch
from diffusers import AutoPipelineForImage2Image
from diffusers.utils import make_image_grid, load_image

pipeline = AutoPipelineForImage2Image.from_pretrained(
    "kandinsky-community/kandinsky-2-2-decoder", torch_dtype=torch.float16, use_safetensors=True
)
pipeline.enable_model_cpu_offload()
# xFormers가 설치되어 있지 않거나 PyTorch 2.0 이상이 설치되어 있는 경우 다음 줄을 제거합니다.
pipeline.enable_xformers_memory_efficient_attention()

# 이미지 준비
url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img-init.png"
init_image = load_image(url)

prompt = "Astronaut in a jungle, cold color palette, muted colors, detailed, 8k"

# 파이프라인에 프롬프트와 이미지 전달
image = pipeline(prompt, image=init_image).images[0]
make_image_grid([init_image, image], rows=1, cols=2)

<div class="flex gap-4">
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img-init.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">초기 이미지</figcaption>
  </div>
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img-kandinsky.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">생성된 이미지</figcaption>
  </div>
</div>

## 파이프라인 매개변수 구성

이미지 생성 프로세스와 이미지 품질에 영향을 미치는 파이프라인에서 구성할 수 있는 몇 가지 중요한 매개변수가 있습니다. 이러한 매개변수가 수행하는 작업과 변경 사항이 출력에 미치는 영향에 대해 자세히 살펴보겠습니다.

### 강도

`strength`는 고려해야 할 가장 중요한 매개변수 중 하나이며 생성된 이미지에 큰 영향을 미칩니다. 생성된 이미지가 초기 이미지와 얼마나 유사한지를 결정합니다. 즉,

- 📈 `strength` 값이 높을수록 모델은 초기 이미지와 다른 이미지를 생성하는 데 더 많은 "창의성"을 발휘합니다. `strength` 값이 1.0이면 초기 이미지가 거의 무시됩니다.
- 📉 `strength` 값이 낮을수록 생성된 이미지가 초기 이미지와 더 유사해집니다.

`strength`와 `num_inference_steps` 매개변수는 관련이 있습니다. `strength`는 추가할 노이즈 단계 수를 결정하기 때문입니다. 예를 들어 `num_inference_steps`가 50이고 `strength`가 0.8이면 초기 이미지에 40(50 * 0.8) 단계의 노이즈를 추가한 다음 40단계 동안 노이즈를 제거하여 새로 생성된 이미지를 얻는다는 의미입니다.

In [ ]:
import torch
from diffusers import AutoPipelineForImage2Image
from diffusers.utils import make_image_grid, load_image

pipeline = AutoPipelineForImage2Image.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
pipeline.enable_model_cpu_offload()
# xFormers가 설치되어 있지 않거나 PyTorch 2.0 이상이 설치되어 있는 경우 다음 줄을 제거합니다.
pipeline.enable_xformers_memory_efficient_attention()

# 이미지 준비
url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img-init.png"
init_image = load_image(url)

prompt = "Astronaut in a jungle, cold color palette, muted colors, detailed, 8k"

# 파이프라인에 프롬프트와 이미지 전달
image = pipeline(prompt, image=init_image, strength=0.8).images[0]
make_image_grid([init_image, image], rows=1, cols=2)

<div class="flex flex-row gap-4">
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img-strength-0.4.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">strength = 0.4</figcaption>
  </div>
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img-strength-0.6.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">strength = 0.6</figcaption>
  </div>
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img-strength-1.0.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">strength = 1.0</figcaption>
  </div>
</div>

### 안내 척도

`guidance_scale` 매개변수는 생성된 이미지와 텍스트 프롬프트가 얼마나 밀접하게 정렬되는지를 제어하는 데 사용됩니다. `guidance_scale` 값이 높을수록 생성된 이미지가 프롬프트와 더 잘 정렬되고, `guidance_scale` 값이 낮을수록 생성된 이미지가 프롬프트에서 더 많이 벗어날 수 있습니다.

`guidance_scale`을 `strength`와 결합하여 모델의 표현력을 더욱 정밀하게 제어할 수 있습니다. 예를 들어, 최대의 창의성을 위해 높은 `strength + guidance_scale`을 결합하거나, 초기 이미지와 유사하지만 프롬프트에 엄격하게 구속되지 않는 이미지를 생성하기 위해 낮은 `strength`와 낮은 `guidance_scale`을 결합할 수 있습니다.

In [ ]:
import torch
from diffusers import AutoPipelineForImage2Image
from diffusers.utils import make_image_grid, load_image

pipeline = AutoPipelineForImage2Image.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
pipeline.enable_model_cpu_offload()
# xFormers가 설치되어 있지 않거나 PyTorch 2.0 이상이 설치되어 있는 경우 다음 줄을 제거합니다.
pipeline.enable_xformers_memory_efficient_attention()

# 이미지 준비
url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img-init.png"
init_image = load_image(url)

prompt = "Astronaut in a jungle, cold color palette, muted colors, detailed, 8k"

# 파이프라인에 프롬프트와 이미지 전달
image = pipeline(prompt, image=init_image, guidance_scale=8.0).images[0]
make_image_grid([init_image, image], rows=1, cols=2)

<div class="flex flex-row gap-4">
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img-guidance-0.1.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">guidance_scale = 0.1</figcaption>
  </div>
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img-guidance-3.0.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">guidance_scale = 5.0</figcaption>
  </div>
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img-guidance-7.5.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">guidance_scale = 10.0</figcaption>
  </div>
</div>

### 부정적 프롬프트

부정적 프롬프트는 모델이 이미지에 특정 요소를 포함하지 않도록 조건을 지정하며, 이미지 품질을 개선하거나 이미지를 수정하는 데 사용할 수 있습니다. 예를 들어, "나쁜 디테일" 또는 "흐릿함"과 같은 부정적 프롬프트를 포함하여 모델이 고품질 이미지를 생성하도록 유도하여 이미지 품질을 개선할 수 있습니다. 또는 이미지에서 제외할 항목을 지정하여 이미지를 수정할 수 있습니다.

In [ ]:
import torch
from diffusers import AutoPipelineForImage2Image
from diffusers.utils import make_image_grid, load_image

pipeline = AutoPipelineForImage2Image.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
pipeline.enable_model_cpu_offload()
# xFormers가 설치되어 있지 않거나 PyTorch 2.0 이상이 설치되어 있는 경우 다음 줄을 제거합니다.
pipeline.enable_xformers_memory_efficient_attention()

# 이미지 준비
url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img-init.png"
init_image = load_image(url)

prompt = "Astronaut in a jungle, cold color palette, muted colors, detailed, 8k"
negative_prompt = "ugly, deformed, disfigured, poor details, bad anatomy"

# 파이프라인에 프롬프트와 이미지 전달
image = pipeline(prompt, negative_prompt=negative_prompt, image=init_image).images[0]
make_image_grid([init_image, image], rows=1, cols=2)

<div class="flex flex-row gap-4">
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img-negative-1.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">negative_prompt = "ugly, deformed, disfigured, poor details, bad anatomy"</figcaption>
  </div>
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img-negative-2.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">negative_prompt = "jungle"</figcaption>
  </div>
</div>

## 연쇄된 이미지-이미지 파이프라인

이미지를 생성하는 것 외에도 이미지-이미지 파이프라인을 사용하는 다른 흥미로운 방법이 있습니다(물론 그것도 꽤 멋지지만요). 한 단계 더 나아가 다른 파이프라인과 연결할 수 있습니다.

### 텍스트-이미지-이미지

텍스트-이미지 및 이미지-이미지 파이프라인을 연결하면 텍스트에서 이미지를 생성하고 생성된 이미지를 이미지-이미지 파이프라인의 초기 이미지로 사용할 수 있습니다. 이는 이미지를 처음부터 완전히 생성하려는 경우에 유용합니다. 예를 들어 Stable Diffusion과 Kandinsky 모델을 연결해 보겠습니다.

텍스트-이미지 파이프라인으로 이미지를 생성하여 시작합니다.

In [ ]:
from diffusers import AutoPipelineForText2Image, AutoPipelineForImage2Image
import torch
from diffusers.utils import make_image_grid

pipeline = AutoPipelineForText2Image.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
pipeline.enable_model_cpu_offload()
# xFormers가 설치되어 있지 않거나 PyTorch 2.0 이상이 설치되어 있는 경우 다음 줄을 제거합니다.
pipeline.enable_xformers_memory_efficient_attention()

text2image = pipeline("Astronaut in a jungle, cold color palette, muted colors, detailed, 8k").images[0]
text2image

이제 이 생성된 이미지를 이미지-이미지 파이프라인에 전달할 수 있습니다.

In [ ]:
pipeline = AutoPipelineForImage2Image.from_pretrained(
    "kandinsky-community/kandinsky-2-2-decoder", torch_dtype=torch.float16, use_safetensors=True
)
pipeline.enable_model_cpu_offload()
# xFormers가 설치되어 있지 않거나 PyTorch 2.0 이상이 설치되어 있는 경우 다음 줄을 제거합니다.
pipeline.enable_xformers_memory_efficient_attention()

image2image = pipeline("Astronaut in a jungle, cold color palette, muted colors, detailed, 8k", image=text2image).images[0]
make_image_grid([text2image, image2image], rows=1, cols=2)

### 이미지-이미지-이미지

여러 이미지-이미지 파이프라인을 함께 연결하여 더 흥미로운 이미지를 만들 수도 있습니다. 이는 이미지에 반복적으로 스타일을 적용하거나, 짧은 GIF를 생성하거나, 이미지의 색상을 복원하거나, 이미지의 누락된 영역을 복원하는 데 유용할 수 있습니다.

이미지를 생성하여 시작합니다.

In [ ]:
import torch
from diffusers import AutoPipelineForImage2Image
from diffusers.utils import make_image_grid, load_image

pipeline = AutoPipelineForImage2Image.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
pipeline.enable_model_cpu_offload()
# xFormers가 설치되어 있지 않거나 PyTorch 2.0 이상이 설치되어 있는 경우 다음 줄을 제거합니다.
pipeline.enable_xformers_memory_efficient_attention()

# 이미지 준비
url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img-init.png"
init_image = load_image(url)

prompt = "Astronaut in a jungle, cold color palette, muted colors, detailed, 8k"

# 파이프라인에 프롬프트와 이미지 전달
image = pipeline(prompt, image=init_image, output_type="latent").images[0]

<Tip>

불필요한 디코딩-인코딩 단계를 피하기 위해 파이프라인에서 `output_type="latent"`를 지정하여 모든 출력을 잠재 공간에 유지하는 것이 중요합니다. 이는 연결된 파이프라인이 동일한 VAE를 사용하는 경우에만 작동합니다.

</Tip>

이 파이프라인의 잠재 출력을 다음 파이프라인에 전달하여 [만화책 아트 스타일](https://huggingface.co/ogkalu/Comic-Diffusion)로 이미지를 생성합니다.

In [ ]:
pipeline = AutoPipelineForImage2Image.from_pretrained(
    "ogkalu/Comic-Diffusion", torch_dtype=torch.float16
)
pipeline.enable_model_cpu_offload()
# xFormers가 설치되어 있지 않거나 PyTorch 2.0 이상이 설치되어 있는 경우 다음 줄을 제거합니다.
pipeline.enable_xformers_memory_efficient_attention()

# 이 체크포인트를 사용하려면 프롬프트에 "charliebo artstyle" 토큰을 포함해야 합니다.
image = pipeline("Astronaut in a jungle, charliebo artstyle", image=image, output_type="latent").images[0]

한 번 더 반복하여 최종 이미지를 [픽셀 아트 스타일](https://huggingface.co/kohbanye/pixel-art-style)로 생성합니다.

In [ ]:
pipeline = AutoPipelineForImage2Image.from_pretrained(
    "kohbanye/pixel-art-style", torch_dtype=torch.float16
)
pipeline.enable_model_cpu_offload()
# xFormers가 설치되어 있지 않거나 PyTorch 2.0 이상이 설치되어 있는 경우 다음 줄을 제거합니다.
pipeline.enable_xformers_memory_efficient_attention()

# 이 체크포인트를 사용하려면 프롬프트에 "pixelartstyle" 토큰을 포함해야 합니다.
image = pipeline("Astronaut in a jungle, pixelartstyle", image=image).images[0]
make_image_grid([init_image, image], rows=1, cols=2)

### 이미지-업스케일러-초해상도

이미지-이미지 파이프라인을 연결하는 또 다른 방법은 업스케일러 및 초해상도 파이프라인과 연결하여 이미지의 세부 수준을 실제로 높이는 것입니다.

이미지-이미지 파이프라인으로 시작합니다.

In [ ]:
import torch
from diffusers import AutoPipelineForImage2Image
from diffusers.utils import make_image_grid, load_image

pipeline = AutoPipelineForImage2Image.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
pipeline.enable_model_cpu_offload()
# xFormers가 설치되어 있지 않거나 PyTorch 2.0 이상이 설치되어 있는 경우 다음 줄을 제거합니다.
pipeline.enable_xformers_memory_efficient_attention()

# 이미지 준비
url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img-init.png"
init_image = load_image(url)

prompt = "Astronaut in a jungle, cold color palette, muted colors, detailed, 8k"

# 파이프라인에 프롬프트와 이미지 전달
image_1 = pipeline(prompt, image=init_image, output_type="latent").images[0]

<Tip>

불필요한 디코딩-인코딩 단계를 피하기 위해 파이프라인에서 `output_type="latent"`를 지정하여 모든 출력을 *잠재* 공간에 유지하는 것이 중요합니다. 이는 연결된 파이프라인이 동일한 VAE를 사용하는 경우에만 작동합니다.

</Tip>

이미지 해상도를 높이기 위해 업스케일러 파이프라인에 연결합니다.

In [ ]:
from diffusers import StableDiffusionLatentUpscalePipeline

upscaler = StableDiffusionLatentUpscalePipeline.from_pretrained(
    "stabilityai/sd-x2-latent-upscaler", torch_dtype=torch.float16, use_safetensors=True
)
upscaler.enable_model_cpu_offload()
upscaler.enable_xformers_memory_efficient_attention()

image_2 = upscaler(prompt, image=image_1).images[0]

마지막으로 초해상도 파이프라인에 연결하여 해상도를 더욱 향상시킵니다.

In [ ]:
from diffusers import StableDiffusionUpscalePipeline

super_res = StableDiffusionUpscalePipeline.from_pretrained(
    "stabilityai/stable-diffusion-x4-upscaler", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
super_res.enable_model_cpu_offload()
super_res.enable_xformers_memory_efficient_attention()

image_3 = super_res(prompt, image=image_2).images[0]
make_image_grid([init_image, image_3.resize((512, 512))], rows=1, cols=2)

## 이미지 생성 제어

원하는 대로 정확하게 보이는 이미지를 생성하는 것은 어려울 수 있으며, 이것이 바로 제어된 생성 기술과 모델이 유용한 이유입니다. `negative_prompt`를 사용하여 부분적으로 이미지 생성을 제어할 수 있지만 프롬프트 가중치 및 ControlNet과 같은 더 강력한 방법이 있습니다.

### 프롬프트 가중치

프롬프트 가중치를 사용하면 프롬프트에서 각 개념의 표현을 조정할 수 있습니다. 예를 들어 "정글 속 우주비행사, 차가운 색상 팔레트, 차분한 색상, 디테일, 8k"와 같은 프롬프트에서 "우주비행사"와 "정글"의 임베딩을 늘리거나 줄이도록 선택할 수 있습니다. [Compel](https://github.com/damian0815/compel) 라이브러리는 프롬프트 가중치를 조정하고 임베딩을 생성하기 위한 간단한 구문을 제공합니다. [프롬프트 가중치](https://huggingface.co/docs/diffusers/main/en/using-diffusers/weighted_prompts) 가이드에서 임베딩을 만드는 방법을 배울 수 있습니다.

[AutoPipelineForImage2Image](https://huggingface.co/docs/diffusers/main/en/api/pipelines/auto_pipeline#diffusers.AutoPipelineForImage2Image)에는 `prompt` 매개변수를 대체하는 임베딩을 전달할 수 있는 `prompt_embeds`(부정 프롬프트를 사용하는 경우 `negative_prompt_embeds`) 매개변수가 있습니다.

In [ ]:
from diffusers import AutoPipelineForImage2Image
import torch

pipeline = AutoPipelineForImage2Image.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
pipeline.enable_model_cpu_offload()
# xFormers가 설치되어 있지 않거나 PyTorch 2.0 이상이 설치되어 있는 경우 다음 줄을 제거합니다.
pipeline.enable_xformers_memory_efficient_attention()

image = pipeline(prompt_embeds=prompt_embeds, # Compel에서 생성됨
    negative_prompt_embeds=negative_prompt_embeds, # Compel에서 생성됨
    image=init_image,
).images[0]

### ControlNet

ControlNet은 추가 조건 이미지 입력을 통합하여 이미지를 생성하는 보다 유연하고 정확한 방법을 제공합니다. 조건 이미지는 캐니 이미지, 깊이 맵, 이미지 분할, 심지어 낙서까지 될 수 있습니다! 어떤 유형의 조건 이미지를 선택하든 ControlNet은 해당 정보를 보존하는 이미지를 생성합니다.

예를 들어, 이미지의 공간 정보를 유지하기 위해 깊이 맵으로 이미지를 조건화해 보겠습니다.

In [ ]:
from diffusers.utils import load_image, make_image_grid

# 이미지 준비
url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img-init.png"
init_image = load_image(url)
init_image = init_image.resize((958, 960)) # 깊이 이미지 크기로 조정
depth_image = load_image("https://huggingface.co/lllyasviel/control_v11f1p_sd15_depth/resolve/main/images/control.png")
make_image_grid([init_image, depth_image], rows=1, cols=2)

깊이 맵에 조건화된 ControlNet 모델과 [AutoPipelineForImage2Image](https://huggingface.co/docs/diffusers/main/en/api/pipelines/auto_pipeline#diffusers.AutoPipelineForImage2Image)를 로드합니다.

In [ ]:
from diffusers import ControlNetModel, AutoPipelineForImage2Image
import torch

controlnet = ControlNetModel.from_pretrained("lllyasviel/control_v11f1p_sd15_depth", torch_dtype=torch.float16, variant="fp16", use_safetensors=True)
pipeline = AutoPipelineForImage2Image.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5", controlnet=controlnet, torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
pipeline.enable_model_cpu_offload()
# xFormers가 설치되어 있지 않거나 PyTorch 2.0 이상이 설치되어 있는 경우 다음 줄을 제거합니다.
pipeline.enable_xformers_memory_efficient_attention()

이제 깊이 맵, 초기 이미지 및 프롬프트에 따라 새 이미지를 생성합니다.

In [ ]:
prompt = "Astronaut in a jungle, cold color palette, muted colors, detailed, 8k"
image_control_net = pipeline(prompt, image=init_image, control_image=depth_image).images[0]
make_image_grid([init_image, depth_image, image_control_net], rows=1, cols=3)

<div class="flex flex-row gap-4">
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img-init.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">초기 이미지</figcaption>
  </div>
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/lllyasviel/control_v11f1p_sd15_depth/resolve/main/images/control.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">깊이 이미지</figcaption>
  </div>
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img-controlnet.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">ControlNet 이미지</figcaption>
  </div>
</div>

ControlNet에서 생성된 이미지에 새로운 [스타일](https://huggingface.co/nitrosocke/elden-ring-diffusion)을 적용하여 이미지-이미지 파이프라인에 연결해 보겠습니다.

In [ ]:
pipeline = AutoPipelineForImage2Image.from_pretrained(
    "nitrosocke/elden-ring-diffusion", torch_dtype=torch.float16,
)
pipeline.enable_model_cpu_offload()
# xFormers가 설치되어 있지 않거나 PyTorch 2.0 이상이 설치되어 있는 경우 다음 줄을 제거합니다.
pipeline.enable_xformers_memory_efficient_attention()

prompt = "elden ring style astronaut in a jungle" # 프롬프트에 "elden ring style" 토큰 포함
negative_prompt = "ugly, deformed, disfigured, poor details, bad anatomy"

image_elden_ring = pipeline(prompt, negative_prompt=negative_prompt, image=image_control_net, strength=0.45, guidance_scale=10.5).images[0]
make_image_grid([init_image, depth_image, image_control_net, image_elden_ring], rows=2, cols=2)

<div class="flex justify-center">
  <img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img-elden-ring.png">
</div>

## 최적화

확산 모델을 실행하는 것은 계산 비용이 많이 들고 집약적이지만 몇 가지 최적화 트릭을 사용하면 소비자 및 무료 계층 GPU에서 완전히 실행할 수 있습니다. 예를 들어 PyTorch 2.0의 [스케일드 닷 프로덕트 어텐션](https://huggingface.co/docs/diffusers/main/en/using-diffusers/../optimization/fp16#scaled-dot-product-attention) 또는 [xFormers](https://huggingface.co/docs/diffusers/main/en/using-diffusers/../optimization/xformers)와 같이 메모리 효율적인 어텐션 형식을 사용할 수 있습니다(둘 중 하나를 사용할 수 있지만 둘 다 사용할 필요는 없습니다). 다른 파이프라인 구성 요소가 CPU에서 대기하는 동안 모델을 GPU로 오프로드할 수도 있습니다.

```diff
+ pipeline.enable_model_cpu_offload()
+ pipeline.enable_xformers_memory_efficient_attention()
```

[`torch.compile`](https://huggingface.co/docs/diffusers/main/en/using-diffusers/../optimization/fp16#torchcompile)을 사용하면 UNet을 래핑하여 추론 속도를 더욱 높일 수 있습니다.

In [ ]:
pipeline.unet = torch.compile(pipeline.unet, mode="reduce-overhead", fullgraph=True)

자세한 내용은 [메모리 사용량 줄이기](https://huggingface.co/docs/diffusers/main/en/using-diffusers/../optimization/memory) 및 [추론 가속화](https://huggingface.co/docs/diffusers/main/en/using-diffusers/../optimization/fp16) 가이드를 참조하세요.